https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/10/06/attention/

딥러닝 모델이 특정 벡터에 주목하게 만들어 모델의 성능을 높이는 기법인 어텐션(attention) 매커니즘에 대해 살펴보도록 하겠습니다. 이 글은 미국 스탠포드 대학의 CS224d 강의와 원 논문을 정리하였음을 먼저 밝힙니다. 

#### 동기
어텐션 매커니즘은 기계번역(machine translation)을 위한 sequence-to-sequence 모델(S2S)에 처음 도입됐습니다. S2S 아키텍처를 간단히 나타낸 그림은 다음과 같습니다. 소스랭귀지(A,B,C)를 입력으로 해서 벡터로 만드는 앞부분을 인코더(encoder), 인코더가 출력한 벡터를 입력으로 해서 타겟랭귀지(W,X,Y,Z)를 출력하는 뒷부분을 디코더(decoder)라고 합니다.

<img src='./imgs/attention.png'>

그런데 여기에서 소스랭귀지와 타겟랭귀지의 길이가 길어질 수록 모델의 성능이 나빠집니다. W를 예측할 때 A,B,C 모두에 집중해 보게 되면 정확도가 떨어질 수 있습니다. 모델로 하여금 ‘중요한 부분만 집중(attention)하게 만들자’가 어텐션 매커니즘의 핵심 아이디어가 되겠습니다.

#### 핵심 아이디어
예컨대 독일어 “Ich mochte ein bier”를 영어 “I’d like a beer”로 번역하는 S2S 모델을 만든다고 칩시다. 모델이 네번째 단어인 ‘beer’를 예측할 때 ‘bier’에 주목하게 만들고자 합니다. 어텐션 매커니즘의 가정은 인코더가 ‘bier’를 받아서 벡터로 만든 결과(인코더 출력)는 디코더가 ‘beer’를 예측할 때 쓰는 벡터(디코더 입력)와 유사할 것이라는 점입니다.

#### 인코더 계산과정
먼저 인코더 계산과정을 살펴보겠습니다. 인코더는 $i$번째 단어벡터 $x_i$를 받아서 그에 해당하는 히든스테이트 벡터 $h_i$를 만듭니다. 이후 $h_i$가 $i$번째 열벡터가 되도록 행렬 형태로 차곡차곡 쌓아놓습니다. 이 행렬을 $F$라고 정의합시다. 아래 그림은 양방향(bi-directional) 모델을 가정한 것입니다.

<img src='./imgs/attention1.png'>


#### 디코더 계산과정

$e_ij$는 디코더가 $i$번째 단어를 예측할 때 쓰는 직전 스텝의 히든스테이트 벡터 $s_{i−1}$이 인코더의 $j$번째 열벡터 $h_j$와 얼마나 유사한지를 나타내는 스코어(스칼라)값입니다. 예컨대 어텐션 매커니즘이 제대로 작동한다면 ‘bier’에 해당하는 디코더 출력 벡터와 ‘beer’를 예측할 때 쓰이는 인코더 입력벡터의 유사도가 높게 나타날 겁니다. 다음과 같이 정의됩니다.

eij=a(si−1,hj)
위 식에서 a는 원 논문에는 alignment model이라 소개돼 있습니다. si−1과 hj 간 유사도를 잘 뽑아낼 수 있다면 다양한 변형이 가능하다고 합니다. 실제로 eij를 구할 때 쓰이는 a는 (1) FTVsi−1 (2) vTtanh(WF+Vsi−1) 등 다양하게 쓰입니다. 여기에서 v,V,W 등은 어텐션을 적용하기 위한 학습 파라메터입니다.

eij에 소프트맥스 함수를 적용해 합이 1이 되도록 확률값으로 변환합니다. Tx는 인코더 입력 단어의 수를 가리킵니다.

αij=exp(eij)∑Txk=1exp(eik)
디코더가 i번째 단어를 예측할 때 쓰이는 attention vector ai는 다음과 같이 정의됩니다.

αi→=[αi1,αi2,...,αiTx]
디코더가 i번째 단어를 예측할 때 쓰이는 context vector ci는 다음과 같이 정의됩니다. 인코더의 j번째 열벡터를 어텐션 확률값으로 가중합을 한 것이라고 볼 수 있겠습니다.

ci→=∑j=1Txαijhj=Fαi→

이번 글에서는 self-attention에 대하여 필자가 연구한 다음 두 논문을 다뤄보고자 합니다.

1. Jongchan Park*, Sanghyun Woo*, Joon-Young Lee, and In So Kweon: “BAM: Bottleneck Attention Module” , in BMVC 2018 (Oral)

2. Jongchan Park*, Sanghyun Woo*, Joon-Young Lee, and In So Kweon: “CBAM: Convolutional Block Attention Module” , in ECCV 2018

두 논문 모두 CNN의 성능 향상을 위한 self-attention module을 제안하고 있습니다. 이번 글에서는 두 연구의 연구 동기 및 방법에 대해서 간략하게 소개하고자 합니다.

### Previous Works and Motivation

CNN에서의 Attention 개념은 주로 Visual Question Answering 또는 Image Captioning처럼 multi-modal 간의 관계를 이용한 feature selection에 많이 사용되었습니다. 가장 기념비적인 논문인 “Show, Attend and Tell: Neural Image Caption Generation with Visual Attention”[1]에서는 아래의 그림과 같이 어떤 부분을 보고(look, attend) 문장을 생성(tell)했는지 효과적으로 시각화할 수 있습니다. 또 다른 논문인 “Dual Attention Networks for Multimodal Reasoning and Matching”[2]에서는 질문과 이미지의 양방향 attention을 통해서 VQA와 cross-modal retrieval의 성능을 향상시켰습니다.

<img src='./imgs/selfattention.png'>
show, Attend and Tell[1] 논문 그림. 밑줄의 단어와 상응하는 이미지 파트는 흰 색으로 칠해져있다.


Attention을 일종의 gating이라고 생각한다면, 위의 예시들은 입력에 따라 dynamic feature selection을 한다고 볼 수 있습니다. 질문에 따라서, 또는 생성되는 문장에 따라서 이미지의 feature map을 다르게 선택하기 때문입니다.

더 일반적인 task인 image classification, detection 등에서도 마찬가지로 입력 영상에 따라서 봐야할 부분이 다를 것 입니다. 일반적인 classification CNN에서 입력 영상에 따라서 서로 다른 computational path를 갖게 한다면 성능 향상을 기대해볼 수 있을 것 입니다. 따라서 본 연구에서는 self-attention을 통한 image classification / detection에서의 성능 향상을 목표로 합니다.

네트워크 아키텍쳐 디자인의 관점으로 보면, 이미 depth / width / cardinality (grouped conv)등 많은 측면이 연구가 되어왔습니다. 그 동안 Attention은 각 task에 특화되어 사용되었고, 일반적인 아키텍쳐로써의 연구가 많이 이루어지지 않았습니다. 최근에 들어서야 Residual Attention Networks (CVPR17)[3], Squeeze-and-Excitation (CVPR18)[4], Non-local Neural Network (CVPR18)[5] 등 일반적인 네트워크 구성요소로써의 attention이 주목을 받고 있습니다. 

Attention은 기존의 네트워크 디자인 요소들과 다소 orthogonal하며, 따라서 complementary하게 네트워크 성능 향상이 가능하다고 가정을 하였습니다. 추후 실험 결과를 보면, 저희가 제안한 attention module은 아주 적은 overhead로도 매우 유의미한 성능 향상을 이끌어냅니다. 그 효과는 여러 벤치마크 (ImageNet classification, COCO detection, VOC detection) 및 여러 아키텍쳐 (ResNet, WideResNet, ResNext, MobileNet, etc)에서 입|증되었습니다.

아래의 섹션에서는 BAM과 CBAM의 방법 및 결과를 나눠서 소개하도록 하겠습니다.

### Method and Results
두 모듈 모두 conv feature를 입력으로 받고, 그에 대한 attention을 계산합니다. Sigmoid로 normalize된 attention map은 conv feature에 element-wise 곱해집니다. Sigmoid 대신 softmax같은 방식이 있겠지만, “가장” 중요한 feature를 찾는 것이 목적이 아니기 때문에 mutually exclusive한 softmax 대신 sigmoid를 사용합니다. 아래의 식에서 F는 conv feature, M(F)는 생성된 attention map을 나타냅니다.

<img src='./imgs/selfattention1.png'>

먼저 떠오르는 직관적인 방법은 3D conv feature모양의 attention을 계산하는 것이고, 이미 Residual Attention Networks[3] (RAN)에서 효과가 입증되었습니다. RAN은 encoder-decoder 방식을 이용해서 3D attention map을 생성하는데, 이러한 방식의 단점은 연산량이 매우 많다는 것 입니다. 이 경우, 동일한 연산량으로 더 많은 층을 쌓거나, 여러 네트워크의 앙상블도 가능할 것 입니다. Attention의 효과를 입증하기 위해서는 적은 연산량으로도 큰 성능 향상을 이끌어내는 것이 중요합니다. 따라서, 제안된 두 모듈은 3D attention map을 channel-wise / spatial-wise로 분해하여 계산합니다. 연산 및 파라미터 overhead는 1~2% 수준으로 매우 미미합니다.

모듈을 디자인함에 있어서 가장 중요했던 철학은 “심플함” 입니다. 아무리 좋은 방법이여도 구현이 어려우면 실제 적용이 쉽지 않습니다. BAM과 CBAM은 아주 단순한 pooling 및 convolution으로 이루어져 있습니다. 또한 self-attention을 모듈화(self-contained) 하여 어떠한 CNN에도 쉽게 끼워넣을 수 있게 하였습니다. 또한 두 모듈 다 기존 네트워크과 함께 end-to-end training이 가능합니다. 논문 내 모든 실험에서 모든 실험에서 attention module과 backbone network를 처음부터 동시에 학습(joint training)하였습니다.